# Executable used by PlugX in Uncommon Location - Sysmon Version
Detects the execution of an executable that is typically used by PlugX for DLL side loading started from an uncommon location

## Rule Content
```
- title: Executable used by PlugX in Uncommon Location - Sysmon Version
  id: aeab5ec5-be14-471a-80e8-e344418305c2
  status: experimental
  description: Detects the execution of an executable that is typically used by PlugX
    for DLL side loading started from an uncommon location
  references:
  - http://www.hexacorn.com/blog/2016/03/10/beyond-good-ol-run-key-part-36/
  - https://countuponsecurity.com/2017/06/07/threat-hunting-in-the-enterprise-with-appcompatprocessor/
  author: Florian Roth
  date: 2017/06/12
  tags:
  - attack.s0013
  - attack.defense_evasion
  - attack.t1073
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection_cammute:
      Image: '*\CamMute.exe'
    filter_cammute:
      Image: '*\Lenovo\Communication Utility\\*'
    selection_chrome_frame:
      Image: '*\chrome_frame_helper.exe'
    filter_chrome_frame:
      Image: '*\Google\Chrome\application\\*'
    selection_devemu:
      Image: '*\dvcemumanager.exe'
    filter_devemu:
      Image: '*\Microsoft Device Emulator\\*'
    selection_gadget:
      Image: '*\Gadget.exe'
    filter_gadget:
      Image: '*\Windows Media Player\\*'
    selection_hcc:
      Image: '*\hcc.exe'
    filter_hcc:
      Image: '*\HTML Help Workshop\\*'
    selection_hkcmd:
      Image: '*\hkcmd.exe'
    filter_hkcmd:
      Image:
      - '*\System32\\*'
      - '*\SysNative\\*'
      - '*\SysWowo64\\*'
    selection_mc:
      Image: '*\Mc.exe'
    filter_mc:
      Image:
      - '*\Microsoft Visual Studio*'
      - '*\Microsoft SDK*'
      - '*\Windows Kit*'
    selection_msmpeng:
      Image: '*\MsMpEng.exe'
    filter_msmpeng:
      Image:
      - '*\Microsoft Security Client\\*'
      - '*\Windows Defender\\*'
      - '*\AntiMalware\\*'
    selection_msseces:
      Image: '*\msseces.exe'
    filter_msseces:
      Image:
      - '*\Microsoft Security Center\\*'
      - '*\Microsoft Security Client\\*'
      - '*\Microsoft Security Essentials\\*'
    selection_oinfo:
      Image: '*\OInfoP11.exe'
    filter_oinfo:
      Image: '*\Common Files\Microsoft Shared\\*'
    selection_oleview:
      Image: '*\OleView.exe'
    filter_oleview:
      Image:
      - '*\Microsoft Visual Studio*'
      - '*\Microsoft SDK*'
      - '*\Windows Kit*'
      - '*\Windows Resource Kit\\*'
    selection_rc:
      Image: '*\rc.exe'
    filter_rc:
      Image:
      - '*\Microsoft Visual Studio*'
      - '*\Microsoft SDK*'
      - '*\Windows Kit*'
      - '*\Windows Resource Kit\\*'
      - '*\Microsoft.NET\\*'
    condition: ( selection_cammute and not filter_cammute ) or ( selection_chrome_frame
      and not filter_chrome_frame ) or ( selection_devemu and not filter_devemu )
      or ( selection_gadget and not filter_gadget ) or ( selection_hcc and not filter_hcc
      ) or ( selection_hkcmd and not filter_hkcmd ) or ( selection_mc and not filter_mc
      ) or ( selection_msmpeng and not filter_msmpeng ) or ( selection_msseces and
      not filter_msseces ) or ( selection_oinfo and not filter_oinfo ) or ( selection_oleview
      and not filter_oleview ) or ( selection_rc and not filter_rc )
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((((((((((((process_path.keyword:*\\CamMute.exe AND (NOT (process_path.keyword:*\\Lenovo\\Communication\ Utility\\*))) OR (process_path.keyword:*\\chrome_frame_helper.exe AND (NOT (process_path.keyword:*\\Google\\Chrome\\application\\*)))) OR (process_path.keyword:*\\dvcemumanager.exe AND (NOT (process_path.keyword:*\\Microsoft\ Device\ Emulator\\*)))) OR (process_path.keyword:*\\Gadget.exe AND (NOT (process_path.keyword:*\\Windows\ Media\ Player\\*)))) OR (process_path.keyword:*\\hcc.exe AND (NOT (process_path.keyword:*\\HTML\ Help\ Workshop\\*)))) OR (process_path.keyword:*\\hkcmd.exe AND (NOT (process_path.keyword:(*\\System32\\* OR *\\SysNative\\* OR *\\SysWowo64\\*))))) OR (process_path.keyword:*\\Mc.exe AND (NOT (process_path.keyword:(*\\Microsoft\ Visual\ Studio* OR *\\Microsoft\ SDK* OR *\\Windows\ Kit*))))) OR (process_path.keyword:*\\MsMpEng.exe AND (NOT (process_path.keyword:(*\\Microsoft\ Security\ Client\\* OR *\\Windows\ Defender\\* OR *\\AntiMalware\\*))))) OR (process_path.keyword:*\\msseces.exe AND (NOT (process_path.keyword:(*\\Microsoft\ Security\ Center\\* OR *\\Microsoft\ Security\ Client\\* OR *\\Microsoft\ Security\ Essentials\\*))))) OR (process_path.keyword:*\\OInfoP11.exe AND (NOT (process_path.keyword:*\\Common\ Files\\Microsoft\ Shared\\*)))) OR (process_path.keyword:*\\OleView.exe AND (NOT (process_path.keyword:(*\\Microsoft\ Visual\ Studio* OR *\\Microsoft\ SDK* OR *\\Windows\ Kit* OR *\\Windows\ Resource\ Kit\\*))))) OR (process_path.keyword:*\\rc.exe AND (NOT (process_path.keyword:(*\\Microsoft\ Visual\ Studio* OR *\\Microsoft\ SDK* OR *\\Windows\ Kit* OR *\\Windows\ Resource\ Kit\\* OR *\\Microsoft.NET\\*)))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()